In [1]:
import sys
sys.path.append('../src_variable')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np

In [2]:
# torch.autograd.set_detect_anomaly(True)

In [14]:
T = 400
network = gk.generate_kvadraturen_small(T)
# network = gk.generate_kvadraturen_wo_tl(T)

ids_bw = ["lundsbro_bw", "elvegata_bw", "tollbod_6bw", "tollbod_5bw", "tollbod_4bw", 
        "tollbod_3bw", "tollbod_2bw", "tollbod_1bw", "v_strand_7bw", "v_strand_6bw",
            "v_strand_5bw", "v_strand_4bw", "v_strand_3bw", "v_strand_2bw", "v_strand_1bw"]            
            

stops_bw = [("tollbod_6bw", 50), ("tollbod_3bw", 90), ("tollbod_1bw", 30), ("v_strand_3bw", 25)]
# stops_bw = [("tollbod_6bw", 50), ("tollbod_3bw", 90)]

times_bw = [40, 130, 190, 250]
# times_bw = [10, 70]

bus_bw = bus.Bus(ids_bw, stops_bw, times_bw, network)

ids_fw = ["v_strand_1fw", "v_strand_2fw", "v_strand_3fw", "h_w_2", "h_w_3",
    "h_w_4", "festning_4fw", "festning_5fw", "festning_6fw", "festning_7fw",
    "tollbod_4fw", "tollbod_5fw", "tollbod_6fw", "elvegata_fw", "lundsbro_fw"]

stops_fw = [("v_strand_1fw", 40), ("h_w_3", 30), ("festning_5fw", 40), ("tollbod_4fw", 25), 
            ("tollbod_6fw", 60)]
# stops_fw = [("h_w_3", 30), ("festning_5fw", 40), ("tollbod_4fw", 25), 
#             ("tollbod_6fw", 60)]
# stops_fw = [("h_w_3", 30), ("festning_5fw", 40), ("tollbod_4fw", 10)]
times_fw = [1, 30, 110, 130, 230]
# times_fw = [10, 40, 80, 120]
# times_fw = [10, 40, 70]

bus_fw = bus.Bus(ids_fw, stops_fw, times_fw, network)

In [15]:
# Create actual network
roads = network.roads
junctions = network.junctions
T = network.T
# Don't store the densities 
# bus_network = nw.RoadNetwork(roads, junctions, T, [bus_fw, bus_bw], store_densities = False)
bus_network = nw.RoadNetwork(roads, junctions, T, [bus_fw, bus_bw], store_densities = False)

In [16]:
# with torch.autograd.set_detect_anomaly(True):
#     densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()
densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

In [17]:
print(bus_delays)

{0: [tensor(2.5116, grad_fn=<AddBackward0>), tensor(26.9507, grad_fn=<AddBackward0>), tensor(5.4994, grad_fn=<AddBackward0>), tensor(66.2066, grad_fn=<AddBackward0>), tensor(24.5655, grad_fn=<AddBackward0>)], 1: [tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0.)]}


In [18]:
# Cloning just to be safe
# objective = bus_delays[0][0].clone() #+ bus_delays[1][0].clone()
# for i in range(len(bus_delays)):
#     for j in range(len(bus_delays[i])):
#         if i == 0 and j == 0:
#             continue
#         objective = objective + bus_delays[i][j].clone()
# objective = bus_delays[0][0] + bus_delays[0][1] + bus_delays[0][2]
objective = torch.tensor(0.0)

# Error with stop number 3 and 4 for fw bus and stop 4 for bw bus

# objective = bus_delays[0][0]  
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        # if j == 2:
        #     continue
        objective = objective + bus_delays[i][j]

In [19]:
print(objective)

tensor(125.7339, grad_fn=<AddBackward0>)


In [20]:
# objective.backward(retain_graph=True)
objective.backward(retain_graph=True)

In [21]:
params = [v for road in bus_network.roads for v in road.Vmax] +\
        [t for junction in bus_network.junctions for traffic_light in junction.trafficlights for t in traffic_light.cycle] +\
        [t for junction in bus_network.junctions for traffic_light in junction.coupled_trafficlights for t in traffic_light.cycle]

In [22]:
print("Road Vmax gradients:")
for road in bus_network.roads:
    for v in road.Vmax:
        print(v.grad)

Road Vmax gradients:
tensor(-0.6195)
tensor(-0.7970)
tensor(-0.7011)
tensor(-0.3300)
tensor(-2.6358e-07)
tensor(-2.6730e-07)
tensor(-2.5743e-07)
tensor(4.2080e-07)
tensor(-3.9667e-10)
tensor(-7.9334e-10)
tensor(-0.0903)
tensor(-1.7202e-09)
tensor(-3.4403e-09)
tensor(-6.8806e-09)
tensor(-1.3761e-08)
tensor(-2.7523e-08)
tensor(-0.0007)
tensor(-2.2197)
tensor(-1.3289)
tensor(-0.9141)
tensor(-1.5138)
tensor(-1.3935)
tensor(-0.8599)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(-0.0178)
tensor(-6.3604e-08)
tensor(-1.2721e-07)
tensor(-7.7155e-08)
tensor(-3.3157e-07)
tensor(-6.6314e-07)
tensor(-1.3263e-06)
tensor(0.0001)
tensor(-2.5025e-06)
tensor(-7.7366e-06)
tensor(-1.5473e-05)
tensor(0.1024)
tensor(-3.5551e-05)
tensor(-0.0294)
tensor(-6.3412e-05)
tensor(-0.2881)
tensor(-0.1039)
tensor(-0.5918)
tensor(-0.8521)
tensor(-0.0008)
tensor(-0.0016)
tensor(-0.0032)
tensor(-0.0064)
tensor(-0.0139)
tensor(0.2581)
tensor(-0.0233)
tensor(-0.0467)
tensor(

In [23]:
for junction in bus_network.junctions:
    for traffic_light in junction.trafficlights:
        for t in traffic_light.cycle:
            print(t.grad)

    for traffic_light in junction.coupled_trafficlights:
        for t in traffic_light.cycle:
            print(t.grad)

tensor(-0.0003)
tensor(-0.0003)
tensor(0.0003)
tensor(1.7719e-06)
tensor(9.0210e-13)
tensor(-1.8135e-15)
tensor(-4.9983e-14)
tensor(-6.8441e-14)
tensor(-5.8331e-07)
tensor(0.)
tensor(6.1885e-09)
tensor(6.1885e-09)
tensor(-0.0014)
tensor(-3.4514e-21)
tensor(-1.0314e-17)
tensor(0.)
tensor(4.8277e-09)
tensor(2.3292e-09)
tensor(-0.0004)
tensor(1.3881e-07)
tensor(1.0005e-05)
tensor(0.)
tensor(0.0002)
tensor(0.0002)
tensor(-0.5324)
tensor(0.0029)


In [24]:
for j in bus_network.junctions:
    print(j.get_activation(195.0, "tollbod_4fw", "tollbod_5fw"))
    # print(j.get_activation(195.0, "festning_7fw", "tollbod_4fw"))

    

(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(True, 1.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
